In [15]:
tips = sc.textFile('../YelpDataset/yelp_academic_dataset_tip.json')
users = sc.textFile('../YelpDataset/yelp_academic_dataset_user.json')
tips.first()

u'{"user_id": "-6rEfobYjMxpUWLNxszaxQ", "text": "Don\'t waste your time.", "business_id": "cE27W9VPgO88Qxe4ol6y_g", "likes": 0, "date": "2013-04-18", "type": "tip"}'

In [16]:
with open('../YelpDataset/negative-words.txt', 'r') as myfile:
    data = myfile.readline()
    negative_words = data.split(",")

negative_words[0:10]

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted']

In [17]:
with open('../YelpDataset/positive-words.txt', 'r') as myfile:
    data = myfile.readline()
    positive_words = data.split(",")

positive_words[0:10]

['',
 'a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed']

In [21]:
users = sc.textFile('../YelpDataset/yelp_academic_dataset_user.json')

import json

def mapper_users_filter(rows):
    for row in rows:
        row = json.loads(row)
        elite = 0
        if len(row['elite']) == 0:
            elite = 1
        else:
            elite = 4 + len(row['elite'])
        yield (row['user_id'], elite)

def mapper_tips_filter(records):
    for row in records:
        row = json.loads(row)
        if (row["type"] == 'tip'):
            text = row["text"].split(" ")
            pos_counter = 0
            neg_counter = 0
            for words in text:
                if words in positive_words:
                    pos_counter += 1
                elif words in negative_words:
                    neg_counter += 1
            if pos_counter > neg_counter:
                yield (row["user_id"], (row["business_id"], 1))  #good
            else:
                yield (row["user_id"], (row["business_id"], -1)) #bad tip
                
tips_filtered = tips.mapPartitions(mapper_tips_filter)
users_filtered = users.mapPartitions(mapper_users_filter)
tips_filtered.take(10)
# joined = users_filtered.join(tips_filtered)


[(u'-6rEfobYjMxpUWLNxszaxQ', (u'cE27W9VPgO88Qxe4ol6y_g', -1)),
 (u'EZ0r9dKKtEGVx2CdnowPCw', (u'mVHrayjG3uZ_RLHkLj-AMg', -1)),
 (u'xb6zEQCw9I-Gl0g06e1KsQ', (u'KayYbHCt-RkbGcPdGOThNg', -1)),
 (u'QawZN4PSW7ng_9SP7pjsVQ', (u'KayYbHCt-RkbGcPdGOThNg', 1)),
 (u'MLQre1nvUtW-RqMTc4iC9A', (u'1_lU0-eSWJCRvNGk78Zh9Q', -1)),
 (u'bvu13GyOUwhEjPum2xjiqQ', (u'1_lU0-eSWJCRvNGk78Zh9Q', -1)),
 (u'bvu13GyOUwhEjPum2xjiqQ', (u'_qopVQ6_Mz6W7-Pmbi56GQ', -1)),
 (u'_QFom7aSHKNCDsNXKd-3xQ', (u'_qopVQ6_Mz6W7-Pmbi56GQ', 1)),
 (u'fvTivrsJoUMYXnOJw9wZfw', (u'wJr6kSA5dchdgOdwH6dZ2w', 1)),
 (u'bvu13GyOUwhEjPum2xjiqQ', (u'Cdcus0NADzyY3XiJM2O5Sg', -1))]